In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
import pprint
import seaborn as sns 
sns.set()
#sns.reset_orig


## Part 1: Getting the Data
##




In [2]:
#######################
import myauz
from myauz.myalpha_funcs import (
    read_data,
    persist_data,
    update_csv,
    compose_portfolio,
    retrieveDF,
    string2date,
    retrievePF,
    initialize_df,
    time_sleep,
    create_path_list,
    refresh_db,
    get_daily_symbol,
)

#import importlib
#importlib.reload(myauz.myalpha_funcs)

from myauz.myalpha_libs_universal import StocksDb

import pprint


from datetime import date
import datetime
import pandas as pd



In [3]:
alphaDB = StocksDb()
alphaDB.check_path()
print("\n", alphaDB.api_key_alpha)


api_key_alpha = alphaDB.api_key_alpha
root_path = alphaDB.path
print('root_path: ',root_path)

read_from_alphavantage = False
rename_column = True


usecols = ["timestamp", "adjusted_close"]
startd = "2015-01-01"
endd = "2019-12-31"

print('usecols:', usecols)
print('startd',startd)
print('endd',endd)


Directory  ../data  already exists

 77K8XPSR3XWWZJ80
root_path:  ..
usecols: ['timestamp', 'adjusted_close']
startd 2015-01-01
endd 2019-12-31


In [4]:
symbol_list = [
   "MSFT",
   "GOOG",    
   "DHR",
   "EMR",
   "ATR",
   "HXGBY",
   "NVS",
   "NVZMY",    
   "CL",
   "MCD",    
   "HEINY",
   "LDSVF",  
]

path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)

bool_list_all = [
   True, #"MSFT", microsoft    
   True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   True, #"DHR", danaher
   True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
   True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
   True, #"HEINY", heineken
   True, #"LDSVF", lindt & spruengli    
]

bool_list_high_tech = [
   True, #"MSFT", microsoft    
   True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   False, #"DHR", danaher
   False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
   False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
   False, #"HEINY", heineken
   False, #"LDSVF", lindt & spruengli    
]

bool_list_industry = [
   False, #"MSFT", microsoft    
   False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   True, #"DHR", danaher
   True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
   False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
   False, #"HEINY", heineken
   False, #"LDSVF", lindt & spruengli    
]

bool_list_consumer = [
   False, #"MSFT", microsoft    
   False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   False, #"DHR", danaher
   False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
   True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
   True, #"HEINY", heineken
   True, #"LDSVF", lindt & spruengli    
]

colors = [
    'magenta',
    'lime',
#   'tab:orange',
    'blue',
    'black',
    'slategrey',
    'tab:brown',
    'tab:olive',
    'darkviolet',
    'red',
    'turquoise',
    'forestgreen',
    'gold',
]

color_dict = dict(zip(symbol_list, colors))
print('\n')
print(color_dict)


{'ATR': '../data/ATR/daily_ATR.csv',
 'CL': '../data/CL/daily_CL.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'EMR': '../data/EMR/daily_EMR.csv',
 'GOOG': '../data/GOOG/daily_GOOG.csv',
 'HEINY': '../data/HEINY/daily_HEINY.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'LDSVF': '../data/LDSVF/daily_LDSVF.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}


{'MSFT': 'magenta', 'GOOG': 'lime', 'DHR': 'blue', 'EMR': 'black', 'ATR': 'slategrey', 'HXGBY': 'tab:brown', 'NVS': 'tab:olive', 'NVZMY': 'darkviolet', 'CL': 'red', 'MCD': 'turquoise', 'HEINY': 'forestgreen', 'LDSVF': 'gold'}


In [5]:
# filter columns(symbols) in dataframe  by bool_list(symbols = True - this is the _included list)
def pf_filtered(_pf, _startd, _included):
    if not (_startd):
        _startd = '2020-01-01'
    if _included is []:
        return _pf
    else: 
        return _pf.loc[startd:,_included]

from itertools import compress
def symbol_list_filtered(_symbol_list,_included):
    #filter the symbol_list
    #symbol_list_filtered = [i for indx,i in enumerate(symbol_list) if included[indx]]
    return list(compress(_symbol_list, _included))

def determine_earliest_notation(_pf):
    """
    input:
    ------
         portfolio notations - portfolio-columns are also symbols
    
    output:
    -------
         dataframe with one row containing earliest notation in time of symbol in portfolio
    
    """
    _pf_tmp = _pf.copy()
    _pf_tmp.fillna(0, inplace=True)
    _symbol_list = list(_pf_tmp.columns)
    _df_aux_0 = _pf_tmp.iloc[0]
    
    for symbol in _symbol_list:
        _df_aux_0[symbol] = (_pf_tmp[_pf_tmp[symbol] > 0].iloc[0]).loc[symbol]  
    return _df_aux_0

In [6]:
print('root_path:',root_path)

root_path: ..


In [7]:
### create dataframe with portfolio adjusted close
usecols = ["timestamp", "adjusted_close"]
path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)
pf = retrievePF(symbol_list, path_list, startd, endd, usecols, rename_column)
print(pf.index.name)
pf.index.rename('DATE', inplace=True)
print(pf.index.name)
pf.fillna(method='ffill', inplace=True)
pf

{'ATR': '../data/ATR/daily_ATR.csv',
 'CL': '../data/CL/daily_CL.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'EMR': '../data/EMR/daily_EMR.csv',
 'GOOG': '../data/GOOG/daily_GOOG.csv',
 'HEINY': '../data/HEINY/daily_HEINY.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'LDSVF': '../data/LDSVF/daily_LDSVF.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}
None
DATE


,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2015-01-02,41.6513,524.81,63.3710,51.3485,61.2037,28.5702,75.3810,39.0758,60.3886,79.6897,32.0285,4789.8804
2015-01-05,41.2638,513.87,62.7426,50.2931,59.3212,28.1195,75.5445,38.0205,59.9425,78.8096,31.2892,4762.0374
2015-01-06,40.6625,501.96,62.3139,50.2100,58.6876,28.0091,74.9071,38.4268,59.3127,78.9548,31.1876,4762.0374
2015-01-07,41.1792,501.10,62.6909,50.1020,59.0274,27.5032,75.2339,37.8897,59.7851,80.3306,31.6496,4762.0374
2015-01-08,42.3906,502.68,63.6594,51.4150,60.7078,27.5032,77.8488,38.9171,60.5985,80.6296,32.2595,4762.0374
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,156.1726,1348.84,152.5790,75.0265,114.2932,56.2600,91.5394,47.9115,67.5017,192.5188,52.9799,7203.6052
2019-12-24,156.1429,1343.56,152.6786,74.3523,113.6592,56.3988,91.4911,47.9115,67.3347,192.9800,53.0195,7203.6052
2019-12-26,157.4227,1360.40,152.3301,74.9092,113.7484,56.2100,91.4815,47.6046,67.3642,193.3627,52.9305,7203.6052


In [8]:
df_returns = (pf/pf.shift(1))-1
#ret = pf.pct_change(1)
pprint.pprint(df_returns.head())

print('___________________')

df1_returns = pf_filtered(df_returns, startd, bool_list_high_tech)


print('\n high_tech:')
#df1_returns = (pf1/pf1.shift(1))-1
pprint.pprint(df1_returns.head(3))


print('\n industry:')
#df2_returns = (pf2/pf2.shift(1))-1
df2_returns = pf_filtered(df_returns, startd, bool_list_industry)
pprint.pprint(df2_returns.head(3))


print('\n consumer: ')
#df3_returns = (pf3/pf3.shift(1))-1
df3_returns = pf_filtered(df_returns, startd, bool_list_consumer)
pprint.pprint(df3_returns.head(3))

                MSFT      GOOG       DHR       EMR       ATR     HXGBY  \
DATE                                                                     
2015-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2015-01-05 -0.009303 -0.020846 -0.009916 -0.020554 -0.030758 -0.015775   
2015-01-06 -0.014572 -0.023177 -0.006833 -0.001652 -0.010681 -0.003926   
2015-01-07  0.012707 -0.001713  0.006050 -0.002151  0.005790 -0.018062   
2015-01-08  0.029418  0.003153  0.015449  0.026207  0.028468  0.000000   

                 NVS     NVZMY        CL       MCD     HEINY     LDSVF  
DATE                                                                    
2015-01-02       NaN       NaN       NaN       NaN       NaN       NaN  
2015-01-05  0.002169 -0.027006 -0.007387 -0.011044 -0.023083 -0.005813  
2015-01-06 -0.008437  0.010686 -0.010507  0.001842 -0.003247  0.000000  
2015-01-07  0.004363 -0.013977  0.007965  0.017425  0.014814  0.000000  
2015-01-08  0.034757  0.027116  0.013605  0

In [9]:
df_cum_returns = initialize_df(symbol_list, startd, endd)
#print(df_cum_returns.head())
#print(df_returns)

df_cum_returns = (1+df_returns).cumprod()
###############achtung dataframe wird auf naechtes datum ohne NaN gesetzt
#df_cum_returns.dropna(inplace=True)
#################
df_cum_returns.head()
#df_cum_returns.head()

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,0.990697,0.979154,0.990084,0.979446,0.969242,0.984225,1.002169,0.972994,0.992613,0.988956,0.976917,0.994187
2015-01-06,0.976260,0.956460,0.983319,0.977828,0.958890,0.980361,0.993713,0.983391,0.982184,0.990778,0.973745,0.994187
2015-01-07,0.988665,0.954822,0.989268,0.975725,0.964442,0.962653,0.998049,0.969646,0.990006,1.008042,0.988170,0.994187
2015-01-08,1.017750,0.957832,1.004551,1.001295,0.991898,0.962653,1.032738,0.995939,1.003476,1.011794,1.007212,0.994187


In [10]:
invested_dict = {
    "MSFT": float(1297.59),
    "GOOG": float(2111),
    "DHR": float(3168.48),
    "EMR": float(418.10),
    "ATR": float(289.49),
    "HXGBY": float(1060.39),
    "NVS": float(404.42),
    "NVZMY": float(1049.95),
    "CL": float(390.03),
    "MCD": float(872.35),  
    "HEINY": float(575.60),     
    "LDSVF": float(6942.25),   
}





def calc_weights(_invested_dict):
    
    _invested_list = list(_invested_dict.values())
    _invested_array =np.array(_invested_list)
    _total_investment=np.sum(_invested_array)
    #print("total investment: ")
    #pprint.pprint(_total_investment)
    _weights_dict = {}
    for symbol, investment in _invested_dict.items():
        _weights_dict[symbol] = investment / _total_investment
        
    return _total_investment, _weights_dict

total_investment, weights_dict = calc_weights(invested_dict)

print("total investment:",total_investment)

weight_sum = 0
for symbol, weight in weights_dict.items():
    pprint.pprint((symbol, weight))
    weight_sum += weight

print(weight_sum)

"""
weights = [
   .1,  #"MSFT",
   .1,  #"GOOG",    
   .1, #"DHR",
   .1,  #"EMR",
   .1,  #"ATR",
   .1,  #"HXGBY",
   .1,  #"NVS",
   .1,  #"NVZMY",    
   .1,  #"CL",
   .05,  #"MCD",    
   .03,  #"HEINY",
   .02,  #"LDSVF",  
]
"""



total investment: 18579.65
('MSFT', 0.0698393134423953)
('GOOG', 0.1136189325417863)
('DHR', 0.17053496702036905)
('EMR', 0.022503114967181836)
('ATR', 0.01558102547679854)
('HXGBY', 0.057072657450490186)
('NVS', 0.021766825532235537)
('NVZMY', 0.05651075235539959)
('CL', 0.020992322245036905)
('MCD', 0.04695190705960554)
('HEINY', 0.030980131487945144)
('LDSVF', 0.373648050420756)
0.9999999999999998


'\nweights = [\n   .1,  #"MSFT",\n   .1,  #"GOOG",    \n   .1, #"DHR",\n   .1,  #"EMR",\n   .1,  #"ATR",\n   .1,  #"HXGBY",\n   .1,  #"NVS",\n   .1,  #"NVZMY",    \n   .1,  #"CL",\n   .05,  #"MCD",    \n   .03,  #"HEINY",\n   .02,  #"LDSVF",  \n]\n'

In [11]:
#ensure that weights vector is in correct order as symbol_list
wlist = []
for symbol in symbol_list:
    print(symbol)
    wlist.append(weights_dict[symbol])

print('wlist',wlist)
#weights = np.array(list(weights_dict.values()))
actual_weights=np.array(wlist)

print('\n',actual_weights.sum())


pf_cum_ret = df_cum_returns * actual_weights * total_investment
pf_cum_ret.fillna(method='ffill', inplace=True, axis=1)
pf_cum_ret['TOTAL_POS'] = pf_cum_ret.sum(axis=1)

print('\n _________________________________________________________________________________')

pf_cum_ret = pf_cum_ret[1:]

pf_cum_ret['TOTAL_POS']
pf_cum_ret.to_csv('../data/pf_cum_ret.csv')

MSFT
GOOG
DHR
EMR
ATR
HXGBY
NVS
NVZMY
CL
MCD
HEINY
LDSVF
wlist [0.0698393134423953, 0.1136189325417863, 0.17053496702036905, 0.022503114967181836, 0.01558102547679854, 0.057072657450490186, 0.021766825532235537, 0.05651075235539959, 0.020992322245036905, 0.04695190705960554, 0.030980131487945144, 0.373648050420756]

 0.9999999999999999

 _________________________________________________________________________________


In [12]:
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')





pf_cum_ret['TOTAL_POS'].plot(color=['royalblue'], figsize=(16,10), linewidth=1.25);


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
pf_cum_ret.head()
color_list = [color_dict.get(x, _) for x in pf_cum_ret.columns]

#pf_cum_ret.fillna(method='ffill', inplace=True)
pf_cum_ret[symbol_list].plot(color=color_list,figsize=(16,10), linewidth=1.25);


plt.title('all stocks in portfolio')
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid( which='minor', linestyle=':', linewidth='0.5', color='gray')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sharpe Ratio

In [14]:
pf_cum_ret['DAILY_RETURN']= pf_cum_ret['TOTAL_POS'].pct_change(1)

print('\nvolatility daily:\t',pf_cum_ret['DAILY_RETURN'].std())
print('return daily: \t\t', pf_cum_ret['DAILY_RETURN'].mean())

actual_annual_volatility = pf_cum_ret['DAILY_RETURN'].std() * 250 **0.5
actual_annual_return = pf_cum_ret['DAILY_RETURN'].mean() * 250

print('\n\nvolatility annually:\t',actual_annual_volatility )
print('return annually:\t', actual_annual_return )
      
cum_return = 100 * ( ( pf_cum_ret['TOTAL_POS'][-1] - pf_cum_ret['TOTAL_POS'][0] ) / pf_cum_ret['TOTAL_POS'][0] )
print('\naccumulated return: \t',cum_return)


#calculate sharpe ratio daily assume risk-free rate to be zero
actual_sharp_ratio = pf_cum_ret['DAILY_RETURN'].mean() / pf_cum_ret['DAILY_RETURN'].std()

# annualize Sharpe Ratio 
# annual sharpe ratio (asr) should be above 1 


print('\nannual sharp ratio should be above 1')
actual_sharp_ratio_annual = (252**0.5)*actual_sharp_ratio

print('\nsharp_ratio:\t\t', actual_sharp_ratio)

print('annual_sharp_ratio:\t',actual_sharp_ratio_annual)

              




volatility daily:	 0.008182320616064968
return daily: 		 0.0005880367086744252


volatility annually:	 0.1293738484625871
return annually:	 0.14700917716860631

accumulated return: 	 100.51984098163533

annual sharp ratio should be above 1

sharp_ratio:		 0.07186674004436937
annual_sharp_ratio:	 1.1408491301659702


In [15]:
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')

pf_cum_ret['DAILY_RETURN'].plot(kind='kde', figsize=(10,8), xlim=[-.05,.05]);
pf_cum_ret


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF,TOTAL_POS,DAILY_RETURN
DATE,,,,,,,,,,,,,,
2015-01-05,1285.517961,2066.994855,3137.060694,409.506512,280.585883,1043.662159,405.297180,1021.594541,387.148788,862.715690,562.313674,6901.895534,18364.293470,NaN
2015-01-06,1266.785271,2019.087975,3115.626168,408.829878,277.588991,1039.564635,401.877521,1032.511648,383.081118,864.305171,560.487771,6901.895534,18271.641680,-0.005045
2015-01-07,1282.882362,2015.628704,3134.475751,407.950499,279.196225,1020.788033,403.630807,1018.080001,386.132193,879.365827,568.790601,6901.895534,18298.816536,0.001487
2015-01-08,1320.621893,2021.984109,3182.899681,418.641470,287.144421,1020.788033,417.659778,1045.685799,391.385675,882.638930,579.751415,6901.895534,18471.096737,0.009415
2015-01-09,1309.521848,1995.798232,3161.460155,412.077881,284.364634,1044.686539,421.386315,1041.169024,387.544058,871.882570,566.382421,6901.895534,18398.169211,-0.003948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,4865.346437,5425.584955,7628.781460,610.895735,540.600298,2088.103737,491.110016,1287.361472,435.971161,2107.471545,952.127962,10440.600604,36873.955384,0.001625
2019-12-24,4864.421173,5404.346640,7633.761350,605.406129,537.601514,2093.255334,490.850886,1287.361472,434.892563,2112.520225,952.839633,10440.600604,36857.857526,-0.000437
2019-12-26,4904.291614,5472.083992,7616.336735,609.940632,538.023425,2086.247975,490.799382,1279.115201,435.083094,2116.709579,951.240170,10440.600604,36940.472405,0.002241


# portfolio optimization
# - monte carlo
# - mathematical optimization by minimizing the negative sharpe ratio

In [16]:
pf


,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2015-01-02,41.6513,524.81,63.3710,51.3485,61.2037,28.5702,75.3810,39.0758,60.3886,79.6897,32.0285,4789.8804
2015-01-05,41.2638,513.87,62.7426,50.2931,59.3212,28.1195,75.5445,38.0205,59.9425,78.8096,31.2892,4762.0374
2015-01-06,40.6625,501.96,62.3139,50.2100,58.6876,28.0091,74.9071,38.4268,59.3127,78.9548,31.1876,4762.0374
2015-01-07,41.1792,501.10,62.6909,50.1020,59.0274,27.5032,75.2339,37.8897,59.7851,80.3306,31.6496,4762.0374
2015-01-08,42.3906,502.68,63.6594,51.4150,60.7078,27.5032,77.8488,38.9171,60.5985,80.6296,32.2595,4762.0374
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,156.1726,1348.84,152.5790,75.0265,114.2932,56.2600,91.5394,47.9115,67.5017,192.5188,52.9799,7203.6052
2019-12-24,156.1429,1343.56,152.6786,74.3523,113.6592,56.3988,91.4911,47.9115,67.3347,192.9800,53.0195,7203.6052
2019-12-26,157.4227,1360.40,152.3301,74.9092,113.7484,56.2100,91.4815,47.6046,67.3642,193.3627,52.9305,7203.6052


In [17]:
# annual mean of returns as %
type(pf.pct_change(1).mean()*252)
(pf.pct_change(1).mean()*252).sort_values(ascending=False)


MSFT     0.292625
GOOG     0.216113
MCD      0.191962
DHR      0.191859
HXGBY    0.175269
ATR      0.140751
HEINY    0.118710
LDSVF    0.113516
EMR      0.098993
NVZMY    0.076070
NVS      0.055104
CL       0.037405
dtype: float64

In [18]:
# we use log returns for algorithms especially for normalizations
log_ret= np.log(pf/pf.shift(1))

log_ret.head()

log_ret.fillna(value=0.000001,inplace=True)
log_ret.to_csv('../data/log_ret2.csv')

In [19]:
(log_ret.mean()).sort_values(ascending=False)

MSFT     0.001052
GOOG     0.000743
MCD      0.000705
DHR      0.000698
HXGBY    0.000535
ATR      0.000500
HEINY    0.000403
LDSVF    0.000325
EMR      0.000298
NVZMY    0.000168
NVS      0.000152
CL       0.000089
dtype: float64

In [20]:
# annual variance
print((log_ret.var()*252).sort_values(ascending=False))

# annual standard deviation
print((log_ret.std()*252**0.5).sort_values(ascending=False))

HXGBY    0.080898
NVZMY    0.068072
LDSVF    0.063107
GOOG     0.056829
MSFT     0.054085
EMR      0.047635
HEINY    0.034335
NVS      0.033757
DHR      0.031545
CL       0.029752
ATR      0.029438
MCD      0.028348
dtype: float64
HXGBY    0.284426
NVZMY    0.260905
LDSVF    0.251211
GOOG     0.238389
MSFT     0.232563
EMR      0.218254
HEINY    0.185296
NVS      0.183731
DHR      0.177608
CL       0.172488
ATR      0.171574
MCD      0.168368
dtype: float64


In [21]:
log_ret.corr() 

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
MSFT,1.000000,0.664300,0.510852,0.468826,0.386638,0.233891,0.367108,0.273803,0.321577,0.382195,0.320917,0.057501
GOOG,0.664300,1.000000,0.437663,0.391439,0.308522,0.207705,0.305488,0.207008,0.253730,0.302562,0.290305,0.030534
DHR,0.510852,0.437663,1.000000,0.483404,0.442515,0.200363,0.421479,0.260888,0.386434,0.394953,0.328556,0.033778
EMR,0.468826,0.391439,0.483404,1.000000,0.377032,0.252782,0.315084,0.273948,0.289304,0.248765,0.251409,0.052468
ATR,0.386638,0.308522,0.442515,0.377032,1.000000,0.169289,0.297592,0.223090,0.386261,0.313881,0.283267,0.075545
HXGBY,0.233891,0.207705,0.200363,0.252782,0.169289,1.000000,0.137860,0.174966,0.124345,0.106831,0.153753,0.064261
NVS,0.367108,0.305488,0.421479,0.315084,0.297592,0.137860,1.000000,0.286412,0.289352,0.263199,0.436422,0.096825
NVZMY,0.273803,0.207008,0.260888,0.273948,0.223090,0.174966,0.286412,1.000000,0.190925,0.158368,0.340666,0.018063
CL,0.321577,0.253730,0.386434,0.289304,0.386261,0.124345,0.289352,0.190925,1.000000,0.382582,0.336946,0.069311
MCD,0.382195,0.302562,0.394953,0.248765,0.313881,0.106831,0.263199,0.158368,0.382582,1.000000,0.219513,0.037878


In [22]:
log_ret.hist(bins=50, figsize=(10,10));

#type(hist)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
np.random.seed(101)
# to keep getting the same random weights

print(pf.columns)
num_weights = len(log_ret.columns)
weights = np.array(np.random.random(num_weights))
pprint.pprint(weights)
print('sum:',np.sum(weights))
# but they do not add up to 1

print('\n rebalance: \n')
weights=weights/np.sum(weights)
print(weights)
np.sum(weights)

Index(['MSFT', 'GOOG', 'DHR', 'EMR', 'ATR', 'HXGBY', 'NVS', 'NVZMY', 'CL',
       'MCD', 'HEINY', 'LDSVF'],
      dtype='object')
array([0.51639863, 0.57066759, 0.02847423, 0.17152166, 0.68527698,
       0.83389686, 0.30696622, 0.89361308, 0.72154386, 0.18993895,
       0.55422759, 0.35213195])
sum: 5.824657602087313

 rebalance: 

[0.08865734 0.09797444 0.00488857 0.02944751 0.11765103 0.14316668
 0.05270116 0.15341899 0.12387747 0.03260946 0.09515196 0.06045539]


1.0

In [24]:
#
print('expected portfolio return')
some_exp_ret = np.sum((log_ret.mean() * weights) * 252)
print((log_ret.mean() * weights) * 252)
some_exp_ret
print(some_exp_ret)
print('\n expected volatility')
#denominator of sharp-ratio

some_exp_vol = np.sqrt(np.dot((weights.T),np.dot(log_ret.cov()*252, weights)))
print(some_exp_vol)

some_sr = some_exp_ret / some_exp_vol
print('\n sharpe ratio: ')
print(some_sr)

expected portfolio return
MSFT     0.023511
GOOG     0.018355
DHR      0.000860
EMR      0.002212
ATR      0.014811
HXGBY    0.019313
NVS      0.002016
NVZMY    0.006479
CL       0.002791
MCD      0.005790
HEINY    0.009654
LDSVF    0.004946
dtype: float64
0.11073708948967811

 expected volatility
0.12528249211161546

 sharpe ratio: 
0.8838991595970274



## determine number of iterations in monte carlo

In [25]:
import numpy as np

np.random.seed(101)

num_pf = 10000

num_weights = len(log_ret.columns)
all_weights = np.zeros((num_pf, num_weights))
ret_arr = np.zeros(num_pf)
vol_arr = np.zeros(num_pf)
sharpe_arr = np.zeros(num_pf)

for ind in range(num_pf):
    _weights = np.array(np.random.random(num_weights))
    _weights = _weights / np.sum(_weights)
    all_weights[ind,:] = _weights
    # expected return
    ret_arr[ind] = np.sum((log_ret.mean() * _weights) * 252)
    #expected volatility
    vol_arr[ind] = np.sqrt(np.dot((_weights.T),np.dot(log_ret.cov()*252, _weights)))
    #sharpe ratio
    sharpe_arr[ind] = ret_arr[ind] / vol_arr[ind]
    
print('done')

done


In [26]:
print(sharpe_arr.max())
mc_sharp_ratio_annual = sharpe_arr.max()
sharpe_arr


1.2398056777993256


array([0.88389916, 0.97015703, 0.89581749, ..., 0.85767534, 1.11595755,
       1.0301339 ])

In [27]:
sharp_argmax = sharpe_arr.argmax()
sharp_argmax

4248

In [28]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr, c= sharpe_arr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('return')



# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')


max_sr_ret = ret_arr[sharp_argmax]
max_sr_vol = vol_arr[sharp_argmax]
plt.scatter(max_sr_vol, max_sr_ret,  c='blue')
plt.scatter(actual_annual_volatility, actual_annual_return,  c='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
log_ret.head()

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2015-01-02,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
2015-01-05,-0.009347,-0.021066,-0.009966,-0.020768,-0.031241,-0.015901,0.002167,-0.027378,-0.007415,-0.011106,-0.023353,-0.005830
2015-01-06,-0.014679,-0.023450,-0.006856,-0.001654,-0.010738,-0.003934,-0.008473,0.010630,-0.010562,0.001841,-0.003252,0.000000
2015-01-07,0.012627,-0.001715,0.006032,-0.002153,0.005773,-0.018227,0.004353,-0.014076,0.007933,0.017275,0.014705,0.000000
2015-01-08,0.028993,0.003148,0.015331,0.025869,0.028070,0.000000,0.034167,0.026754,0.013514,0.003715,0.019087,0.000000


# Machine Learning Approach 

In [30]:
from scipy.optimize import minimize 
import numpy as np


In [31]:
#helper functions

def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights)  * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252,weights)))
    sr = ret / vol
    return np.array([ret,vol,sr])

#minimize negative sharpe ratio which is the same as maximize sr
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1

def check_sum(weights):
    #return 0 if the sum of the weights is 1
    return np.sum(weights) - 1

def minimize_volatility(weights):
    return get_ret_vol_sr(weights)[1]

In [32]:
cons = ({'type': 'eq', 'fun':check_sum})
#defines a constraint for minimize function - it says
#{Constraint, dict} or List of {Constraint, dict}
# type will be equations and a function

bounds = ((0,1),(0,1), (0,1),(0,1),
          (0,1),(0,1), (0,1),(0,1),
          (0,1),(0,1), (0,1),(0,1))
#bounds : sequence or `Bounds`, optional
#        Bounds on variables for L-BFGS-B, TNC, SLSQP and
#        trust-constr methods. There are two ways to specify the bounds:
#    
#           1. Instance of `Bounds` class.
#           2. Sequence of ``(min, max)`` pairs for each element in `x`. None
#               is used to specify no bound.



#sum of weights is 1 
init_guess = [0.25,0.25,0.25,0.25, 0.25,0.25,0.25,0.25, 0.25,0.25,0.25,0.25]

In [33]:
opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds,constraints=cons )
opt_results

     fun: -1.3942436762102572
     jac: array([ 1.62571669e-04,  8.60993862e-02,  4.44397330e-04,  5.97633049e-01,
        9.66399908e-04,  3.21418047e-05,  5.74195445e-01,  5.13400987e-01,
        6.77824959e-01, -2.22206116e-04, -1.88386440e-03, -6.61373138e-04])
 message: 'Optimization terminated successfully.'
    nfev: 115
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([2.61884943e-01, 2.46156903e-16, 1.74943169e-01, 0.00000000e+00,
       5.11421308e-02, 5.24566909e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.49651495e-01, 2.05648629e-02, 8.93567085e-02])

In [34]:
if (np.sum(opt_results.x)) == 1:
    print("ok")

get_ret_vol_sr(opt_results.x)

array([0.18520675, 0.13283672, 1.39424368])

In [35]:
# create an array of possible returns  y-values
frontier_y = np.linspace(0,0.25,100)
frontier_y

array([0.        , 0.00252525, 0.00505051, 0.00757576, 0.01010101,
       0.01262626, 0.01515152, 0.01767677, 0.02020202, 0.02272727,
       0.02525253, 0.02777778, 0.03030303, 0.03282828, 0.03535354,
       0.03787879, 0.04040404, 0.04292929, 0.04545455, 0.0479798 ,
       0.05050505, 0.0530303 , 0.05555556, 0.05808081, 0.06060606,
       0.06313131, 0.06565657, 0.06818182, 0.07070707, 0.07323232,
       0.07575758, 0.07828283, 0.08080808, 0.08333333, 0.08585859,
       0.08838384, 0.09090909, 0.09343434, 0.0959596 , 0.09848485,
       0.1010101 , 0.10353535, 0.10606061, 0.10858586, 0.11111111,
       0.11363636, 0.11616162, 0.11868687, 0.12121212, 0.12373737,
       0.12626263, 0.12878788, 0.13131313, 0.13383838, 0.13636364,
       0.13888889, 0.14141414, 0.14393939, 0.14646465, 0.1489899 ,
       0.15151515, 0.1540404 , 0.15656566, 0.15909091, 0.16161616,
       0.16414141, 0.16666667, 0.16919192, 0.17171717, 0.17424242,
       0.17676768, 0.17929293, 0.18181818, 0.18434343, 0.18686

In [36]:
frontier_volatility = []
#for a possible return what is the risk
#in order to get the return/volatility combinations on the efficient frontier 
#we add the second constraint
for possible_return in frontier_y:
    cons = ({'type':'eq', 'fun':check_sum},{'type':'eq', 'fun':lambda w: (get_ret_vol_sr(w)[0]-possible_return)})
    result = minimize(minimize_volatility, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
    frontier_volatility.append(result['fun'])
    
    
frontier_volatility
    

[0.18321732051908202,
 0.18190760824867797,
 0.1806177519007784,
 0.17935051312972777,
 0.17810886942515156,
 0.1768960274089075,
 0.1757154363664948,
 0.17457080188755297,
 0.1734660994611118,
 0.1709909326388449,
 0.15482679349893175,
 0.143787007349401,
 0.13763645941712652,
 0.1329967218420932,
 0.12939621301141288,
 0.12675379303242562,
 0.1247375532608542,
 0.12315724269302564,
 0.12178282553414435,
 0.12050946165050579,
 0.11933130144855886,
 0.11825140064766852,
 0.11724539160896945,
 0.11628633787572618,
 0.11537510313054958,
 0.1145127850270182,
 0.11370054650921085,
 0.11293947875078776,
 0.11223119414877707,
 0.11157549020267861,
 0.11097356696439667,
 0.11042684414532225,
 0.10993632864097605,
 0.10950209044185057,
 0.1091262090264545,
 0.10880633207513021,
 0.10854358637229783,
 0.10832581407788369,
 0.10815147875892865,
 0.10801926396032156,
 0.10793018018173978,
 0.10788127883934864,
 0.10787418469859447,
 0.10790826663968303,
 0.10798380374468808,
 0.10810031244269531,

In [37]:
frontier_sr = frontier_y / frontier_volatility
print('\nmax sharp ratio calculated:\t\t',frontier_sr.max())

print('number of argument in array frontier_sr:\t',frontier_sr.argmax())

max_sr_ret_calc = frontier_y[frontier_sr.argmax()]
max_sr_vol_calc = frontier_volatility[frontier_sr.argmax()]
ml_sharp_ratio_annual = max_sr_ret_calc / max_sr_vol_calc

print('\n\nmax return calculated:\t\t',frontier_volatility[frontier_sr.argmax()])
print('max volatility calculated:\t',frontier_y[frontier_sr.argmax()])


max sharp ratio calculated:		 1.394199138333909
number of argument in array frontier_sr:	 73


max return calculated:		 0.13222173882830526
max volatility calculated:	 0.18434343434343436


In [52]:
# calculated efficient frontier
plt.figure(figsize=(14,8))
plt.plot(frontier_volatility,frontier_y, 'g--');

# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')


#scatter-plot from monte carlo simulation
plt.scatter(vol_arr,ret_arr, c= sharpe_arr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('return')

# actual portfolio Volatility/Return - red
print('\n\nactual x-value of Pf:\t\t', actual_annual_volatility)
print('actual  y-value of Pf:\t\t', actual_annual_return)
plt.scatter(actual_annual_volatility, actual_annual_return,  c='red')
plt.axhline(actual_annual_return, color='silver',linestyle='--')
plt.axvline(actual_annual_volatility, color='silver', linestyle='--');

# Monte-Carlo Volatility/Return  - blue
print('\nmonte_carlo x-value-max:\t', max_sr_vol)
print('monte_carlo y-value-max:\t', max_sr_ret)
plt.scatter(max_sr_vol, max_sr_ret,  c='blue');
plt.axhline(max_sr_ret, color='lightsteelblue',linestyle='--')
plt.axvline(max_sr_vol, color='lightsteelblue', linestyle='--');



# Machine_Learning calculated Volatility/Return - green
print('\nMachine_L x-value-max:\t\t', max_sr_vol_calc)
print('Machine_L y-value-max:\t\t', max_sr_ret_calc)
plt.scatter(max_sr_vol_calc, max_sr_ret_calc,  c='green');
plt.axhline(max_sr_ret_calc, color='gray',linestyle='--')
plt.axvline(max_sr_vol_calc, color='gray', linestyle='--');

# sharpe-Ratios
print('\n\nactual sharp-ratio-annual:\t',actual_sharp_ratio_annual)
print('monte_carlo sharp_ratio-annual:\t',mc_sharp_ratio_annual)
print('Machine_L sharp_ratio-annual:\t',ml_sharp_ratio_annual)

print('\noverall return improvement:\t',max_sr_ret_calc - actual_annual_return )
print('return improvement to actual:\t',(max_sr_ret_calc - actual_annual_return)/actual_annual_return )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …



actual x-value of Pf:		 0.1293738484625871
actual  y-value of Pf:		 0.14700917716860631

monte_carlo x-value-max:	 0.13064958707191224
monte_carlo y-value-max:	 0.16198009985389417

Machine_L x-value-max:		 0.13222173882830526
Machine_L y-value-max:		 0.18434343434343436


actual sharp-ratio-annual:	 1.1408491301659702
monte_carlo sharp_ratio-annual:	 1.2398056777993256
Machine_L sharp_ratio-annual:	 1.394199138333909

overall return improvement:	 0.03733425717482805
return improvement to actual:	 0.25395868403514027


In [39]:
import pandas as pd

actual_weights
mc_weights = all_weights[sharp_argmax,:]
ml_weights = np.round((opt_results.x),decimals=6)

#ml_weights = opt_results.x

print(ml_weights)


w_data = {'symbol': symbol_list,
          'actual_weights': actual_weights,
          'mc_weights': mc_weights,
          'ml_weights': ml_weights,
        }
w_data

df_weights_compare = pd.DataFrame(w_data, columns = ['symbol','actual_weights','mc_weights','ml_weights'])
df_weights_compare['diff_%'] = (ml_weights - actual_weights) / actual_weights * 100

print('\n',df_weights_compare)



[0.261885 0.       0.174943 0.       0.051142 0.052457 0.       0.
 0.       0.349651 0.020565 0.089357]

    symbol  actual_weights  mc_weights  ml_weights      diff_%
0    MSFT        0.069839    0.159770    0.261885  274.982209
1    GOOG        0.113619    0.108728    0.000000 -100.000000
2     DHR        0.170535    0.180344    0.174943    2.584826
3     EMR        0.022503    0.045107    0.000000 -100.000000
4     ATR        0.015581    0.120407    0.051142  228.232568
5   HXGBY        0.057073    0.067734    0.052457   -8.087336
6     NVS        0.021767    0.039158    0.000000 -100.000000
7   NVZMY        0.056511    0.002533    0.000000 -100.000000
8      CL        0.020992    0.012341    0.000000 -100.000000
9     MCD        0.046952    0.151079    0.349651  644.700315
10  HEINY        0.030980    0.097114    0.020565  -33.618745
11  LDSVF        0.373648    0.015683    0.089357  -76.085249


In [48]:
print('\nact. sharp-ratio-annual:\t',actual_sharp_ratio_annual)
print('mc sharp-ratio-annual:\t\t',mc_sharp_ratio_annual)
print('ml sharp-ratio-annual:\t\t',ml_sharp_ratio_annual)
print('difference: \t\t\t',(ml_sharp_ratio_annual-actual_sharp_ratio_annual))


act. sharp-ratio-annual:	 1.1408491301659702
mc sharp-ratio-annual:		 1.2398056777993256
ml sharp-ratio-annual:		 1.394199138333909
difference: 			 0.25335000816793873
